In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\curso\appdata\local\temp\pip-req-build-goq158ti
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\curso\AppData\Local\Temp\pip-req-build-goq158ti'

[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install opencv-python pycocotools matplotlib onnxruntime onnx


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install torch


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install torchvision


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install matplotlib


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install opencv-python


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install h5py


[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import torchvision
import os
import h5py

In [19]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [20]:
#Modelo
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [21]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [22]:
 # Function to convert list of boolean arrays to uint8 arrays
def bool_array_to_uint8(array):
    return array.astype(np.uint8)

In [23]:
# Ruta de la carpeta donde se encuentran los archivos
carpeta = "bandej"  # Reemplaza esto con la ruta de tu carpeta

# Crear la carpeta "segmentación" si no existe
general_folder_name = "bandeja"
if not os.path.exists(general_folder_name):
    os.makedirs(general_folder_name)

# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta)



In [24]:
# Ejemplo de cómo cambiar los nombres de los archivos
for i, nombre_archivo in enumerate(archivos):
    # Verifica si el elemento es un archivo (no un directorio)
    if os.path.isfile(os.path.join(carpeta, nombre_archivo)):
        # Cambiar el nombre del archivo (por ejemplo, agregar '_nuevo' al final del nombre)
        nuevo_nombre = f"{i + 1}.jpg"
        # Ruta completa del archivo original
        ruta_original = os.path.join(carpeta, nombre_archivo)
        # Ruta completa del archivo con el nuevo nombre
        ruta_nueva = os.path.join(general_folder_name, nuevo_nombre)
        # Renombrar el archivo
        os.rename(ruta_original, ruta_nueva)

print("Nombres de archivos cambiados correctamente.")

Nombres de archivos cambiados correctamente.


In [25]:
# Crear la carpeta "segmentación" si no existe
general_folder_name = "Segmentos"
if not os.path.exists(general_folder_name):
    os.makedirs(general_folder_name)
    
# Crear la carpeta "Matrices" si no existe
name = "Matrices"
if not os.path.exists(name):
    os.makedirs(name)
             
# Obtener la lista de nombres de archivos en la carpeta "Prueba"
folder_path = 'bandeja'
file_names = os.listdir(folder_path)

# Leer cada imagen, convertirla a formato RGB y agregarla a la lista "images"
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    image = cv2.imread(file_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Crear la carpeta para la imagen actual si no existe
    nombre_carpeta_imagen = os.path.join(general_folder_name, file_name[:-4])
    if not os.path.exists(nombre_carpeta_imagen):
            os.makedirs(nombre_carpeta_imagen)
            
    # Guardar la imagen original en la carpeta
    ruta_absoluta = os.path.abspath(os.path.join(nombre_carpeta_imagen, file_name))
    cv2.imwrite(ruta_absoluta, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
    
    mask = mask_generator_2.generate(image_rgb)
    
    # Supongamos que tienes la lista 'masks2' como se menciona en la pregunta
    
    # Nombre del archivo .h5
    nombre_archivo = file_name[:-4] +'.h5'

    # Crear el archivo HDF5 en la carpeta de destino
    ruta_archivo_h5 = os.path.join(name, nombre_archivo)

    with h5py.File(ruta_archivo_h5, 'w') as hf:
        for idx, obj in enumerate(mask):
        # Obtener la matriz de segmentación de la máscara
            segmentation = obj['segmentation']

            # Crear una imagen en color del mismo tamaño que la matriz de segmentación
            imagen_segmentada = np.zeros(segmentation.shape + (3,), dtype=np.uint8)

            # Obtener los colores correspondientes de la imagen original
            colores_originales = image_rgb[segmentation]

            # Establecer los píxeles correspondientes a la segmentación con los colores originales
            imagen_segmentada[segmentation] = colores_originales

            # Guardar la imagen segmentada en la carpeta correspondiente
            nombre_imagen_segmentada = f"segmentacion_{idx + 1}.jpg"
            cv2.imwrite(os.path.join(nombre_carpeta_imagen, nombre_imagen_segmentada), cv2.cvtColor(imagen_segmentada, cv2.COLOR_RGB2BGR))

            
            #ARCHIVO
            group = hf.create_group(nombre_imagen_segmentada[:-4])
            group.create_dataset('segmentation', data=bool_array_to_uint8(obj['segmentation']))
            group.create_dataset('area', data=obj['area'])
            group.create_dataset('bbox', data=obj['bbox'])
            group.create_dataset('predicted_iou', data=obj['predicted_iou'])
            group.create_dataset('point_coords', data=obj['point_coords'])
            group.create_dataset('stability_score', data=obj['stability_score'])
            group.create_dataset('crop_box', data=obj['crop_box'])

    
            
            
    
    
    
    
    

In [18]:
# Función para convertir uint8 arrays a boolean arrays
def uint8_array_to_bool(array):
    return array.astype(bool)

# Leer el archivo HDF5
with h5py.File('1.h5', 'r') as hf:
    # Obtener las claves (nombres de los grupos) dentro del archivo HDF5
    group_keys = list(hf.keys())

    # Acceder a los datos de cada grupo dentro del archivo
    for key in group_keys:
        group = hf[key]
        segmentation = uint8_array_to_bool(group['segmentation'][:])
        area = group['area'][()]
        bbox = group['bbox'][:]
        predicted_iou = group['predicted_iou'][()]
        point_coords = group['point_coords'][:]
        stability_score = group['stability_score'][()]
        crop_box = group['crop_box'][:]

        # Ahora puedes trabajar con los datos obtenidos de cada grupo
        print(f"Datos del grupo {key}:")
        print("Segmentation:")
        print(segmentation)
        print("Area:", area)
        print("Bbox:", bbox)
        print("Predicted IoU:", predicted_iou)
        print("Point Coords:", point_coords)
        print("Stability Score:", stability_score)
        print("Crop Box:", crop_box)
        print("\n")

Datos del grupo segmentacion_1:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 14928
Bbox: [ 771 1480  127  119]
Predicted IoU: 1.0005661249160767
Point Coords: [[ 873.375 1568.625]]
Stability Score: 0.9907345771789551
Crop Box: [ 396  596  804 1004]


Datos del grupo segmentacion_10:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 8623
Bbox: [670 997 134 102]
Predicted IoU: 0.9790168404579163
Point Coords: [[ 722.625 1066.625]]
Stability Score: 0.9789087772369385
Crop Box: [ 396  596  804 1004]


Datos del grupo segmenta

Datos del grupo segmentacion_117:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 158174
Bbox: [439 730 588 598]
Predicted IoU: 0.9978060126304626
Point Coords: [[ 772.875 1254.875]]
Stability Score: 0.9719851016998291
Crop Box: [ 396  596  804 1004]


Datos del grupo segmentacion_118:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 10330
Bbox: [ 585 1088  148  121]
Predicted IoU: 0.9231060743331909
Point Coords: [[ 672.375 1192.125]]
Stability Score: 0.9371277093887329
Crop Box: [ 396  596  804 1004]


Datos del grupo seg

Datos del grupo segmentacion_138:
Segmentation:
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 277748
Bbox: [   0    0  766 1599]
Predicted IoU: 0.9947869777679443
Point Coords: [[ 206.25 1575.  ]]
Stability Score: 0.9645478129386902
Crop Box: [   0    0 1200 1600]


Datos del grupo segmentacion_139:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 364647
Bbox: [ 173  264  876 1033]
Predicted IoU: 0.9913558959960938
Point Coords: [[618.75 725.  ]]
Stability Score: 0.9777107238769531
Crop Box: [   0    0 1200 1600]


Datos del grupo segm

Datos del grupo segmentacion_18:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 1079
Bbox: [822 804  36  42]
Predicted IoU: 0.9572524428367615
Point Coords: [[823.125 815.625]]
Stability Score: 0.9779816269874573
Crop Box: [ 396  596  804 1004]


Datos del grupo segmentacion_19:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 1433
Bbox: [655 901  42  48]
Predicted IoU: 0.9563202857971191
Point Coords: [[672.375 941.125]]
Stability Score: 0.9765193462371826
Crop Box: [ 396  596  804 1004]


Datos del grupo segmentacion_2:


Datos del grupo segmentacion_38:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 3289
Bbox: [453 203  72  92]
Predicted IoU: 0.9802801012992859
Point Coords: [[521.625   282.65625]]
Stability Score: 0.9806879758834839
Crop Box: [ 396    0  804 1005]


Datos del grupo segmentacion_39:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 5093
Bbox: [455 204  85 122]
Predicted IoU: 0.9775390625
Point Coords: [[521.625   282.65625]]
Stability Score: 0.9763014912605286
Crop Box: [ 396    0  804 1005]


Datos del grupo segmentacion_4

Datos del grupo segmentacion_67:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 132
Bbox: [665 838  10  20]
Predicted IoU: 0.8782832026481628
Point Coords: [[672.375   847.96875]]
Stability Score: 0.9411764740943909
Crop Box: [ 396    0  804 1005]


Datos del grupo segmentacion_68:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 5170
Bbox: [ 336 1177  168   47]
Predicted IoU: 0.9820196032524109
Point Coords: [[ 377.34375 1192.125  ]]
Stability Score: 0.9869431853294373
Crop Box: [   0  596  805 1004]


Datos del grupo seg

Datos del grupo segmentacion_82:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 1991
Bbox: [565 553  43  68]
Predicted IoU: 0.9472264051437378
Point Coords: [[578.59375 596.71875]]
Stability Score: 0.9722359776496887
Crop Box: [   0    0  805 1005]


Datos del grupo segmentacion_83:
Segmentation:
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Area: 749
Bbox: [676 902  21  44]
Predicted IoU: 0.9327921271324158
Point Coords: [[679.21875 910.78125]]
Stability Score: 0.9581699371337891
Crop Box: [   0    0  805 1005]


Datos del grupo segmentac